In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import mne
from mne.datasets import fetch_fsaverage

In [2]:
cases = {1: ('A0R0', 'p200309', ['8B2-3', '8B3-4', '8B4-5', '8B7-8', '9C4-5']), #fixed!
        2: ('M0S0', 'p200723', ['6HC6-7', '6HC7-8', '6HC8-9', '8NA4-5', '11HC2-3']), #fixed!
        3: ('M0R0', 'p201209', ['1TP2-3', '2NA5-6', '3HT7-8']), #fixed!
        4: ('M0K0', 'p210831', ['1TP2-3', '2HC3-4', '3HT2-3', '3HT3-4', '3HT4-5']), #fixed!
        5: ('Z0M0', 'p220602', []), #fixed!
        6: ('E0G0', 'p200924', ['1AA1-2', '1AA2-3', '1AA3-4', '4DD1-2', '7GG2-3', '7GG3-4', '11KK6-7', '15OO5-6'])} #fixed!

In [3]:
CASE = cases[1][1]
MEG_CASE = cases[1][0]
seeg_mask = cases[1][2]
VOL_SOURCES = 8194

localization_file = fr'Z:\Diane_Vrubel\Data\sEEG\iEEG_localization.xlsx'
mat_file = fr'Z:\Diane_Vrubel\Data\sEEG\erf\erf_{MEG_CASE}.mat'

src_meg = fr'Z:\Diane_Vrubel\Data\MEG\{MEG_CASE}\forward_model\src.pckl'
trans_meg = mne.read_trans(fr'Z:\Diane_Vrubel\Data\MEG\{MEG_CASE}\forward_model\checked_visually_trans.fif')
iz_meg_file = fr'Z:\Diane_Vrubel\Data\MEG_predictions\{MEG_CASE}_pred_nobin_8.out'
subjects_dir_meg = fr'Z:\Diane_Vrubel\Data\FreeSurfer'

## sEEG

In [4]:
# paths to mne datasets - sample sEEG and FreeSurfer's fsaverage subject
# which is in MNI space
misc_path = mne.datasets.misc.data_path()
sample_path = mne.datasets.sample.data_path()
subjects_dir = sample_path / "subjects"

# use mne-python's fsaverage data
fetch_fsaverage(subjects_dir=subjects_dir, verbose=True)  # downloads if needed

0 files missing from root.txt in C:\Users\CCDM\mne_data\MNE-sample-data\subjects
0 files missing from bem.txt in C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage


MNEWindowsPath('C:/Users/CCDM/mne_data/MNE-sample-data/subjects/fsaverage')

In [5]:
erf_data = sio.loadmat(mat_file)

In [6]:
erf_data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Dec  8 14:09:59 2022',
 '__version__': '1.0',
 '__globals__': [],
 'erf_all': array([[(array([[-1.14796459e+02, -1.03549583e+02, -1.02416468e+02, ...,
                  9.19591814e+01,  8.78581271e+01,  8.97267365e+01],
                [ 2.67577766e+01,  1.68086382e+01,  2.22954055e+01, ...,
                  8.33315087e+01,  8.49802911e+01,  7.71036798e+01],
                [-1.09830655e+01, -1.28075488e+01, -1.89552371e+01, ...,
                 -1.66054715e+01, -1.71154637e+01, -1.63267612e+01],
                ...,
                [-5.87345688e+00, -5.76923305e+00, -6.92990110e+00, ...,
                  9.72809567e+00,  4.50037927e+00,  7.27855361e+00],
                [ 1.17951393e+01,  1.77460899e+00, -6.87884040e+00, ...,
                 -9.80773244e-02,  5.94994773e+00,  8.63074088e-01],
                [-6.35320682e+01, -5.73537613e+01, -4.98253150e+01, ...,
                  4.02983573e+01,  3.56356273

In [7]:
spike_rate = erf_data['erf_all'][0][0][2][0]  # Spike rate 
#hfo_rate = erf_data['erf_all'][0][0][4][0]  # HFO rate 
electrodes = erf_data['erf_all'][0][0][1][0]  # names

localization_df = pd.read_excel(localization_file, sheet_name=CASE) 
localization_df.columns = ['Electrode', 'X', 'Y', 'Z', 'X_2', 'Y_2', 'Z_2', 'Region'] 
region_data = localization_df[['Electrode', 'Region']].set_index('Electrode')['Region'].to_dict()

In [8]:
electrodes

array([array(['1TP1-2'], dtype='<U6'), array(['1TP2-3'], dtype='<U6'),
       array(['1TP3-4'], dtype='<U6'), array(['1TP4-5'], dtype='<U6'),
       array(['1TP5-6'], dtype='<U6'), array(['1TP6-7'], dtype='<U6'),
       array(['1TP7-8'], dtype='<U6'), array(['2B1-2'], dtype='<U5'),
       array(['2B2-3'], dtype='<U5'), array(['2B3-4'], dtype='<U5'),
       array(['2B4-5'], dtype='<U5'), array(['2B5-6'], dtype='<U5'),
       array(['2B6-7'], dtype='<U5'), array(['2B7-8'], dtype='<U5'),
       array(['2B8-9'], dtype='<U5'), array(['2B9-10'], dtype='<U6'),
       array(['3C1-2'], dtype='<U5'), array(['3C2-3'], dtype='<U5'),
       array(['3C3-4'], dtype='<U5'), array(['3C4-5'], dtype='<U5'),
       array(['3C5-6'], dtype='<U5'), array(['3C6-7'], dtype='<U5'),
       array(['3C7-8'], dtype='<U5'), array(['3C8-9'], dtype='<U5'),
       array(['3C9-10'], dtype='<U6'), array(['4GC1-2'], dtype='<U6'),
       array(['4GC2-3'], dtype='<U6'), array(['4GC3-4'], dtype='<U6'),
       array(['4GC4-5'

In [9]:
electrodes[2][0]

'1TP3-4'

In [10]:
# saving coordinates to a dictionary
electrode_coords = {}
for index, row in localization_df.iterrows():
    electrode_coords[row['Electrode']] = (row['X_2'], row['Y_2'], row['Z_2'])

In [11]:
spike_data = [] # a list of dictionaries with electrodes coords and their spike_rate

for i, electrode in enumerate(electrodes):
    electrode_name = electrode[0]  # Access the string and split it
   # if electrode_name in electrode_coords:
    coords_1 = electrode_coords[electrode_name.split('-')[0]]
    if electrode_name.split('-')[0][:(-1)] + str(int(electrode_name.split('-')[0][-1]) + 1) in electrode_coords:
        second_electrode = electrode_name.split('-')[0][:(-1)] + str(int(electrode_name.split('-')[0][-1]) + 1)
    elif electrode_name.split('-')[0][:(-1)] + electrode_name.split('-')[1][-1] in electrode_coords:
        second_electrode = electrode_name.split('-')[0][:(-1)] + electrode_name.split('-')[1][-1]
    else:
        second_electrode = electrode_name.split('-')[0]
    coords_2 = electrode_coords[second_electrode]
    spike_data.append({
            'Electrode': electrode_name,
            'X': (coords_1[0]+coords_2[0])/2,
            'Y': (coords_1[1]+coords_2[1])/2,
            'Z': (coords_1[2]+coords_2[2])/2,
            'Spike Rate': spike_rate[i]
        })
   # else:
    #    print(f"Electrode {electrode_name} not found in coordinates")

spike_df = pd.DataFrame(spike_data)
print("Spike DataFrame:")
print(spike_df)

Spike DataFrame:
   Electrode      X      Y      Z  Spike Rate
0     1TP1-2 -19.20  -1.50 -37.30       36.16
1     1TP2-3 -24.10  -1.60 -38.05       27.54
2     1TP3-4 -29.25  -1.65 -38.30        8.68
3     1TP4-5 -34.65  -1.65 -37.75        2.64
4     1TP5-6 -39.80  -1.60 -36.75        1.24
..       ...    ...    ...    ...         ...
84   12CR3-4  16.05  33.10  11.75        4.74
85   12CR4-5  21.25  33.80  14.10        2.36
86   12CR5-6  26.55  34.95  16.75        0.18
87   12CR6-7  31.60  36.25  19.05        0.10
88   12CR7-8  36.50  37.55  21.00        0.00

[89 rows x 5 columns]


In [12]:
spike_df.head()

,Electrode,X,Y,Z,Spike Rate
0,1TP1-2,-19.20,-1.50,-37.30,36.16
1,1TP2-3,-24.10,-1.60,-38.05,27.54
2,1TP3-4,-29.25,-1.65,-38.30,8.68
3,1TP4-5,-34.65,-1.65,-37.75,2.64
4,1TP5-6,-39.80,-1.60,-36.75,1.24


In [13]:
spike_df[spike_df.Electrode.isin(seeg_mask)]

,Electrode,X,Y,Z,Spike Rate
59,8B2-3,26.30,-14.15,-31.35,12.16
60,8B3-4,31.45,-14.00,-31.45,15.88
61,8B4-5,36.65,-13.80,-31.55,21.60
64,8B7-8,51.40,-13.15,-31.85,2.40
70,9C4-5,36.20,-36.00,-11.85,3.88


In [14]:
spike_df = spike_df[~spike_df.Electrode.isin(seeg_mask)]

In [15]:
spike_df.Electrode.values

array(['1TP1-2', '1TP2-3', '1TP3-4', '1TP4-5', '1TP5-6', '1TP6-7',
       '1TP7-8', '2B1-2', '2B2-3', '2B3-4', '2B4-5', '2B5-6', '2B6-7',
       '2B7-8', '2B8-9', '2B9-10', '3C1-2', '3C2-3', '3C3-4', '3C4-5',
       '3C5-6', '3C6-7', '3C7-8', '3C8-9', '3C9-10', '4GC1-2', '4GC2-3',
       '4GC3-4', '4GC4-5', '4GC5-6', '4GC6-7', '4GC7-8', '4GC8-9',
       '4GC9-10', '5CC1-2', '5CC2-3', '5CC3-4', '5CC4-5', '5CC5-6',
       '5CC6-7', '5CC7-8', '5CC8-9', '5CC9-10', '6CR1-2', '6CR2-3',
       '6CR3-4', '6CR4-5', '6CR5-6', '6CR6-7', '6CR7-8', '7TP1-2',
       '7TP2-3', '7TP3-4', '7TP4-5', '7TP5-6', '7TP6-7', '7TP7-8',
       '7TP8-9', '7TP9-10', '8B5-6', '8B6-7', '8B8-9', '8B9-10', '9C1-2',
       '9C2-3', '9C3-4', '9C5-6', '9C6-7', '9C7-8', '9C8-9', '9C9-10',
       '10GC2-3', '10GC3-4', '10GC4-5', '10GC5-6', '10GC6-7', '10GC7-8',
       '10GC8-9', '12CR2-3', '12CR3-4', '12CR4-5', '12CR5-6', '12CR6-7',
       '12CR7-8'], dtype=object)

In [16]:
# finding the electrode with the highest spike activity

max_spike_electrode = spike_df.loc[spike_df['Spike Rate'].idxmax()]

print(f"Electrode with max spike rate: {max_spike_electrode['Electrode']}")
print(max_spike_electrode['Spike Rate'])
print(f"Localization: X={max_spike_electrode['X']}, Y={max_spike_electrode['Y']}, Z={max_spike_electrode['Z']}")

Electrode with max spike rate: 7TP2-3
40.1
Localization: X=18.0, Y=-6.5, Z=-26.8


In [17]:
MAX_SPIKE = max_spike_electrode['Spike Rate']

In [18]:
ch_names=list(electrode_coords.keys())
coords_mni_ras = np.array(localization_df[['X_2','Y_2','Z_2']])

montage = mne.channels.make_dig_montage(
    ch_pos=dict(zip(ch_names, coords_mni_ras/1000.0)),
    coord_frame='head'  # because MNI is in MRI space
)

info = mne.create_info(ch_names=ch_names, ch_types='seeg', sfreq=1000)
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: 1TP1, 1TP2, 1TP3, 1TP4, 1TP5, 1TP6, 1TP7, 1TP8, 2B1, 2B2, 2B3, ...
 chs: 114 sEEG
 custom_ref_applied: False
 dig: 117 items (3 Cardinal, 114 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 114
 projs: []
 sfreq: 1000.0 Hz
>

In [19]:
info

<Info | 8 non-empty values
 bads: []
 ch_names: 1TP1, 1TP2, 1TP3, 1TP4, 1TP5, 1TP6, 1TP7, 1TP8, 2B1, 2B2, 2B3, ...
 chs: 114 sEEG
 custom_ref_applied: False
 dig: 117 items (3 Cardinal, 114 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 114
 projs: []
 sfreq: 1000.0 Hz
>

In [20]:
identity_trans = mne.transforms.Transform('head', 'mri')  # head → mri
identity_trans['trans'] = np.eye(4)

view_kwargs = dict(azimuth=105, elevation=100, focalpoint=(0, 0, -15))
'''
brain = mne.viz.Brain(
    "fsaverage",
    subjects_dir=subjects_dir,
    cortex="low_contrast",
    alpha=0.25,
    background="white",
    # surf = 'inflated'
)

brain.add_sensors(info, trans=identity_trans)
brain.add_head(alpha=0.25, color="tan")
brain.show_view(distance=400, **view_kwargs)
'''

'\nbrain = mne.viz.Brain(\n    "fsaverage",\n    subjects_dir=subjects_dir,\n    cortex="low_contrast",\n    alpha=0.25,\n    background="white",\n    # surf = \'inflated\'\n)\n\nbrain.add_sensors(info, trans=identity_trans)\nbrain.add_head(alpha=0.25, color="tan")\nbrain.show_view(distance=400, **view_kwargs)\n'

In [21]:
# get standard fsaverage volume (5mm grid) source space
fname_src = subjects_dir / "fsaverage" / "bem" / "fsaverage-vol-5-src.fif"
vol_src = mne.read_source_spaces(fname_src)

aver_src = vol_src

    Reading a source space...
    [done]
    1 source spaces read


In [22]:
import numpy as np
import nibabel as nib
from scipy.spatial import cKDTree

In [23]:
# electrode MNI coordinates (the highest spike rate)
mni_coords_mm = np.array([[15.2, -6.6, -26.6]]) / 1000

# all source points from the source space
all_rr = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in vol_src])

tree = cKDTree(all_rr)
dist, idx = tree.query(mni_coords_mm) # the index of the closest source point

In [24]:
def find_closest(coords, tree):
    mni_coords_mm = np.array([coords]) # coords is a list of x y z
    dist, idx = tree.query(mni_coords_mm) # the index of the closest source point
    return idx

all_rr = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in vol_src])
tree = cKDTree(all_rr)

In [26]:
coordinates = [[x / 1000, y / 1000, z / 1000] for x, y, z in zip(spike_df.X.values, spike_df.Y.values, spike_df.Z.values)]
spike_rates = spike_df["Spike Rate"].values
src_coords = []
rates = dict()

for electrode, rate in zip(coordinates, spike_rates):
    src_id = find_closest(electrode, tree)
    src_coord = all_rr[src_id]
    if str(src_id) in rates:
        rates[str(src_id)].append(rate)
    else:
        rates[str(src_id)] = [rate]
    src_coords.append(src_coord)  

rates_ = dict([(src_id, sum(rate)/len(rate)) for src_id, rate in rates.items()])

In [27]:
all_rr.shape

(14629, 3)

In [28]:
all_rr_mm = all_rr 
source_coords_mm = all_rr_mm[idx]

In [29]:
source_coords_mm

array([[ 0.015, -0.005, -0.025]])

In [30]:
mni_coords_mm 

array([[ 0.0152, -0.0066, -0.0266]])

In [31]:
n_sources = all_rr.shape[0]
data = np.zeros((n_sources, 100))
for idx in rates.keys():
    idx_ = int(idx.replace('[', '').replace(']', ''))
    data[idx_] = np.array([rates_[idx]] * 100)

vertices = [np.arange(n_sources)]

In [32]:
idxs = [int(x.replace('[', '').replace(']', '')) for x in rates.keys()]

In [33]:
rates_tresh = dict()
rates_tresh_ = dict()
for key, value in rates.items():
    if value[0] / MAX_SPIKE > 0.5:
        rates_tresh[key] = value[0] / MAX_SPIKE * 3
        rates_tresh_[key] = value[0] / MAX_SPIKE * 3
    else:
        rates_tresh[key] = [0.0]
        rates_tresh_[key] = 0.0
    '''
    if value[0] >= 20:
        rates_tresh[key] = value[0]
        rates_tresh_[key] = value[0]
    else:
        rates_tresh[key] = [0.0]
        rates_tresh_[key] = 0.0
    rates_tresh[key] = [value[0] / 40.1]
    rates_tresh_[key] = value[0] / 40.1
    '''

In [34]:
unique_sorted_indices = np.array(sorted(idxs))
unique_spike_values = [rates_tresh_[f'[{idx}]'] for idx in unique_sorted_indices]
# Создаем данные для VolSourceEstimate
correct_vertices = np.array([vol_src[0]['vertno'][x] for x in sorted(idxs)])
vertices = [correct_vertices]  # Используем уникальные отсортированные индексы
data = np.array(unique_spike_values)[:, np.newaxis]  # Приведение к форме (n_vertices, n_times)

# Проверяем размерность и корректность
print(f"Data shape: {data.shape}, Vertices: {vertices}")

Data shape: (84, 1), Vertices: [array([ 9777,  9778,  9779, 11047, 11048, 11049, 11061, 11062, 11063,
       11067, 12186, 12187, 12188, 12189, 12216, 12217, 12218, 12247,
       12248, 12267, 12268, 12270, 12304, 12331, 12332, 12333, 13614,
       13616, 13617, 15996, 15997, 15998, 15999, 16000, 17279, 17280,
       17281, 18518, 18519, 18520, 18521, 18522, 18523, 18524, 18525,
       19813, 22887, 22888, 23613, 23614, 24105, 24135, 24136, 24137,
       24165, 24166, 24167, 24176, 24177, 24871, 24872, 24874, 24889,
       24890, 24891, 24892, 24893, 24894, 24895, 24902, 24903, 24929,
       24930, 25260, 25261, 25465, 25499, 26545, 26546, 26577, 27862,
       27863, 29147, 29148], dtype=int64)]


In [35]:
vals_to_verts = dict(zip(vertices[0].tolist(), unique_spike_values))

all_vertices = vol_src[0]['vertno']
all_values = []
for vert in all_vertices:
    if vert in vals_to_verts.keys():
        all_values.append(vals_to_verts[vert])
    else:
        all_values.append(0)
        
full_data = np.array(all_values)[:, np.newaxis]
full_verts = [all_vertices]

full_stc_seeg = mne.VolSourceEstimate(
    data=full_data,
    vertices=full_verts,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [36]:
iz_seeg = []
for act, vert in zip(data.tolist(), vertices[0].tolist()):
    if act[0] != 0:
        iz_seeg.append(vert)

In [37]:
len(iz_seeg)

8

In [38]:
vol_stc = mne.VolSourceEstimate(
    data=data,
    vertices=vertices,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [39]:
vol_stc

<VolSourceEstimate | 84 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (84, 1), ~2 kB>

In [40]:
mne.viz.set_3d_options(antialias=False)

In [41]:
clim = dict(kind="value", lims=[0.5, 0.7, 0.95])

brain = vol_stc.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)


Using pyvistaqt 3d backend.


In [42]:
aseg_file = fr'{subjects_dir}\fsaverage\mri\aparc+aseg.mgz'
label_names = mne.get_volume_labels_from_aseg(aseg_file)

In [43]:
stc_to_label = mne.extract_label_time_course(
    full_stc_seeg,
    (aseg_file, label_names),  
    vol_src,
    mode='max', 
    mri_resolution=True, 
    allow_empty=True 
)

peak_time_idx = np.argmax(np.max(stc_to_label, axis=0))
peak_activations = stc_to_label[:, peak_time_idx]

active_label_indices = np.where(peak_activations > 0)[0]
active_regions = [label_names[i] for i in active_label_indices]

regions_seeg = []
for region in active_regions:
    if ('Unknown' not in region) and ('unknown' not in region) and region != 'Left-Cerebral-White-Matter' and region != 'Right-Cerebral-White-Matter':
        regions_seeg.append(region)

print(len(regions_seeg))
print("Active volume regions:")
for region in regions_seeg:
    print(region)

Reading atlas C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\mri\aparc+aseg.mgz
114/114 atlas regions had at least one vertex in the source space
Extracting time courses for 114 labels (mode: max)
11
Active volume regions:
Left-Amygdala
Right-Inf-Lat-Vent
Right-Hippocampus
Right-Amygdala
ctx-lh-entorhinal
ctx-lh-fusiform
ctx-lh-parahippocampal
ctx-lh-temporalpole
ctx-rh-entorhinal
ctx-rh-fusiform
ctx-rh-inferiortemporal


In [44]:
brain = mne.viz.Brain(
    "fsaverage",
    alpha=0.1,
    cortex="low_contrast",
    subjects_dir=subjects_dir
)
brain.add_volume_labels(aseg="aparc+aseg", labels=regions_seeg)

    Smoothing by a factor of 0.9


## Forward Model

In [45]:
src_vol = mne.read_source_spaces(subjects_dir / "fsaverage" / "bem" / "fsaverage-vol-5-src.fif")
bem = mne.read_bem_solution(subjects_dir / "fsaverage" / "bem" / "fsaverage-5120-5120-5120-bem-sol.fif")

    Reading a source space...
    [done]
    1 source spaces read
Loading surfaces...

Loading the solution matrix...

Three-layer model surfaces loaded.
Loaded linear collocation BEM solution from C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\bem\fsaverage-5120-5120-5120-bem-sol.fif


In [46]:
src_vol[0]['vertno'].shape

(14629,)

In [47]:
montage = mne.channels.make_standard_montage('standard_1020')
ch_names = montage.ch_names[:64]
info = mne.create_info(ch_names=ch_names, sfreq=1000, ch_types='eeg')
info.set_montage(montage)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fpz, Fp2, AF9, AF7, AF5, AF3, AF1, AFz, AF2, AF4, AF6, AF8, ...
 chs: 64 EEG
 custom_ref_applied: False
 dig: 67 items (3 Cardinal, 64 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 64
 projs: []
 sfreq: 1000.0 Hz
>

In [48]:
fwd_vol = mne.make_forward_solution(
    info=info,
    trans='fsaverage',  
    src=src_vol,
    bem=bem,
    eeg=True,
    meg=False
)

Source space          : <SourceSpaces: [<volume, shape=(33, 39, 34), n_used=14629>] MRI (surface RAS) coords, subject 'fsaverage', ~75.3 MB>
MRI -> head transform : C:\Users\CCDM\AppData\Roaming\Python\Python312\site-packages\mne\data\fsaverage\fsaverage-trans.fif
Measurement data      : instance of Info
Conductor model   : instance of ConductorModel
Accurate field computations
Do computations in head coordinates
Free source orientations

Read 1 source spaces a total of 14629 active source locations

Coordinate transformation: MRI (surface RAS) -> head
    0.999994 0.003552 0.000202      -1.76 mm
    -0.003558 0.998389 0.056626      31.09 mm
    -0.000001 -0.056626 0.998395      39.60 mm
    0.000000 0.000000 0.000000       1.00

Read  64 EEG channels from info
Head coordinate coil definitions created.
Source spaces are now in head coordinates.

Employing the head->MRI coordinate transform with the BEM model.
BEM model instance of ConductorModel is now set up

Source spaces are in head

In [49]:
fwd_vol

EEG,64
Source space,Volume with 14629 grid points
Source orientation,Free


In [50]:
fwd_vol['sol']['data'].shape

(64, 43887)

In [51]:
43887 / 3

14629.0

In [52]:
src_vol[0]['vertno'].shape

(14629,)

In [53]:
gain = fwd_vol['sol']['data']
n_sources = gain.shape[1] // 3
gain_fixed = np.zeros((gain.shape[0], n_sources))

for i in range(n_sources):
    gx = gain[:, 3*i + 0]
    gy = gain[:, 3*i + 1]
    gz = gain[:, 3*i + 2]
    gain_fixed[:, i] = np.sqrt(gx**2 + gy**2 + gz**2)

In [54]:
gain_fixed.shape

(64, 14629)

In [55]:
gain_fixed

array([[ 25.66122137,  26.26112968,  26.11235142, ...,  60.94318054,
         58.45786092,  57.82289234],
       [ 25.8924623 ,  25.65646267,  27.20036452, ...,  60.99349681,
         60.31826157,  61.39757567],
       [ 27.17136452,  26.29430384,  28.64182359, ...,  58.0100535 ,
         59.05480123,  61.60677049],
       ...,
       [ 41.82768958,  37.5309054 ,  43.7187873 , ...,  93.2850517 ,
         91.41974824,  92.13214515],
       [ 35.18164841,  32.71404373,  35.33556244, ..., 124.17134565,
        123.80726169, 124.84835877],
       [ 33.20345408,  32.72126499,  31.47880338, ..., 142.07277228,
        142.46080405, 143.65414271]])

In [76]:
full_data.shape

(14629, 1)

In [80]:
for elem in full_data:
    if elem != [0]:
        print(elem / 10)

[0.20603491]
[0.27052369]
[0.15411471]
[0.23236908]
[0.17925187]
[0.23042394]
[0.3]
[0.18389027]


In [116]:
source_data = full_data * (10 ** (-10)) / 3 
# a completely arbitrary coefficient with does not set the entire brain ablaze

In [117]:
sensor_data = gain_fixed @ source_data

In [118]:
sensor_data

array([[5.24819373e-08],
       [5.48613412e-08],
       [5.57025599e-08],
       [4.67284953e-08],
       [4.98904440e-08],
       [5.04174479e-08],
       [4.97908152e-08],
       [4.91171030e-08],
       [4.96414954e-08],
       [5.09866475e-08],
       [5.33206227e-08],
       [5.55509344e-08],
       [5.58143061e-08],
       [5.13303166e-08],
       [4.67298754e-08],
       [4.81891281e-08],
       [4.57763134e-08],
       [4.22926260e-08],
       [4.01278686e-08],
       [4.01646204e-08],
       [4.27347443e-08],
       [4.75657201e-08],
       [5.31008459e-08],
       [5.64040833e-08],
       [5.35085732e-08],
       [4.87608420e-08],
       [4.81079364e-08],
       [4.01782218e-08],
       [3.40355895e-08],
       [3.06689459e-08],
       [3.03998532e-08],
       [3.32645567e-08],
       [3.91613628e-08],
       [4.80152175e-08],
       [5.85125807e-08],
       [5.75449737e-08],
       [5.07704379e-08],
       [4.88312802e-08],
       [3.68369495e-08],
       [2.85998497e-08],


In [119]:
evoked = mne.EvokedArray(sensor_data, info=info, tmin=0)

In [70]:
from mne import make_ad_hoc_cov
noise_cov = make_ad_hoc_cov(info)

In [71]:
inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)

Computing inverse operator with 64 channels.
    64 out of 64 channels remain after picking
Selected 64 channels
Creating the depth weighting matrix...
    64 EEG channels
    limit = 14630/14629 = 3.882442
    scale = 61579.2 exp = 0.8
Whitening the forward solution.
Computing rank from covariance with rank=None
    Using tolerance 5.7e-18 (2.2e-16 eps * 64 dim * 0.0004  max singular value)
    Estimated rank (eeg): 64
    EEG: rank 64 computed from 64 data channels with 0 projectors
    Setting small EEG eigenvalues to zero (without PCA)
Creating the source covariance matrix
Adjusting source covariance matrix.
Computing SVD of whitened and weighted lead field matrix.


C:\Users\CCDM\AppData\Local\Temp\ipykernel_5172\2053878520.py:1: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)
C:\Users\CCDM\AppData\Local\Temp\ipykernel_5172\2053878520.py:1: RuntimeWarning: No average EEG reference present in info["projs"], covariance may be adversely affected. Consider recomputing covariance using with an average eeg reference projector added.
  inverse_operator = mne.minimum_norm.make_inverse_operator(info, fwd_vol, noise_cov=noise_cov)


    largest singular value = 4.72127
    scaling factor to adjust the trace = 5.96068e+22 (nchan = 64 nzero = 0)


In [120]:
snr = 3.0
lambda2 = 1.0 / snr ** 2
evoked.set_eeg_reference(projection=True)
stc_cortex = mne.minimum_norm.apply_inverse(evoked, inverse_operator, lambda2=lambda2, method='dSPM')

EEG channel type selected for re-referencing
Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Preparing the inverse operator for use...
    Scaled noise and source covariance from nave = 1 to nave = 1
    Created the regularized inverter
    The projection vectors do not apply to these channels.
    Created the whitener using a noise covariance matrix with rank 64 (0 small eigenvalues omitted)
    Computing noise-normalization factors (dSPM)...
[done]
Applying inverse operator to ""...
    Picked 64 channels from the data
    Computing inverse...
    Eigenleads need to be weighted ...
    Computing residual...
    Explained  97.5% variance
    Combining the current components...
    dSPM...
[done]


In [121]:
stc_cortex.data

array([[0.34493656],
       [0.34209918],
       [0.34913104],
       ...,
       [0.06893157],
       [0.06417938],
       [0.06248325]])

In [122]:
brain = stc_cortex.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)

## MEG IZ

In [752]:
import pickle

with open(src_meg, 'rb') as f:
    src = pickle.load(f)
    
mne_coords = np.concatenate([s['rr'][s['inuse'].astype(bool)] for s in src])

In [753]:
#vol_mne = mne_coords[VOL_SOURCES::]

In [754]:
identity_trans = mne.transforms.Transform('head', 'mri')  # head → mri
identity_trans['trans'] = np.eye(4)

In [755]:
mni_coords = mne.head_to_mni(mne_coords, MEG_CASE, trans_meg, subjects_dir=subjects_dir_meg )

In [756]:
#mni_vol = mne.head_to_mri(vol_mne, MEG_CASE, trans_meg, subjects_dir=subjects_dir_meg)

In [757]:
activity = np.loadtxt(iz_meg_file)
data = activity[:, np.newaxis]

In [758]:
vol_activity = activity[VOL_SOURCES::]

In [759]:
src_coords = []

for source, rate in zip(mni_coords/1000, activity):
    src_id = find_closest(source, tree)
    src_coord = all_rr[src_id]
    if rate == 1 and src_id.tolist()[0] not in src_coords:
        src_coords.append(src_id.tolist()[0])
correct_vertices = np.array([vol_src[0]['vertno'][x] for x in sorted(src_coords)])
vertices = [correct_vertices]  # Используем уникальные отсортированные индексы

In [760]:
data = np.ones((vertices[0].shape[0],1))

In [761]:
iz_meg = vertices[0].tolist()

In [762]:
all_vertices = vol_src[0]['vertno']
all_values_meg = []
for vert in all_vertices:
    if vert in correct_vertices.tolist():
        all_values_meg.append(3)
    else:
        all_values_meg.append(0)
full_data_meg = np.array(all_values_meg)[:, np.newaxis]
full_verts_meg = [all_vertices]

full_stc_meg = mne.VolSourceEstimate(
    data=full_data_meg,
    vertices=full_verts_meg,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [763]:
meg_to_label = mne.extract_label_time_course(
    full_stc_meg,
    (aseg_file, label_names),  
    vol_src,
    mode='max', 
    mri_resolution=True, 
    allow_empty=True 
)

peak_time_idx = np.argmax(np.max(meg_to_label, axis=0))
peak_activations = meg_to_label[:, peak_time_idx]

active_label_indices = np.where(peak_activations > 0)[0]
active_regions_meg = [label_names[i] for i in active_label_indices]

regions_meg = []
for region in active_regions_meg:
    if ('Unknown' not in region) and ('unknown' not in region) and region != 'Left-Cerebral-White-Matter' and region != 'Right-Cerebral-White-Matter':
        regions_meg.append(region)

print(len(regions_meg))
print("Active volume regions:")
for region in regions_meg:
    print(region)

Reading atlas C:\Users\CCDM\mne_data\MNE-sample-data\subjects\fsaverage\mri\aparc+aseg.mgz
114/114 atlas regions had at least one vertex in the source space
Extracting time courses for 114 labels (mode: max)
17
Active volume regions:
Left-Inf-Lat-Vent
Left-Pallidum
Left-Hippocampus
Left-Amygdala
Left-VentralDC
Right-Inf-Lat-Vent
Right-Hippocampus
Right-Amygdala
Right-VentralDC
ctx-lh-entorhinal
ctx-lh-middletemporal
ctx-lh-parahippocampal
ctx-lh-parsopercularis
ctx-lh-parstriangularis
ctx-lh-precentral
ctx-lh-rostralmiddlefrontal
ctx-lh-superiortemporal


In [764]:
brain = mne.viz.Brain(
    "fsaverage",
    alpha=0.1,
    cortex="low_contrast",
    subjects_dir=subjects_dir
)
brain.add_volume_labels(aseg="aparc+aseg", labels=regions_meg)

    Smoothing by a factor of 0.9


In [765]:
iz_stc = mne.VolSourceEstimate(
    data=data,
    vertices=vertices,
    tmin=0,
    tstep=1,
    subject="fsaverage"
)

In [766]:
iz_stc

<VolSourceEstimate | 152 vertices, subject : fsaverage, tmin : 0.0 (ms), tmax : 0.0 (ms), tstep : 1000.0 (ms), data shape : (152, 1), ~3 kB>

In [ ]:

clim = dict(kind="value", lims=[0.5, 0.7, 0.95])

brain = iz_stc.plot_3d(
    src=vol_src,
    subjects_dir=subjects_dir,
    view_layout="horizontal",
    views=["axial", "coronal", "sagittal"],
    size=(800, 300),
    show_traces=0.4,
    clim=clim,
    add_data_kwargs=dict(colorbar_kwargs=dict(label_font_size=8)),
)


## Metrics

In [767]:
interc = []

In [768]:
for region in regions_seeg:
    if region in regions_meg:
        interc.append(region)

for region in regions_meg:
    if region in regions_seeg and region not in interc:
        interc.append(region)

In [769]:
for region in interc:
    print(region)

Left-Inf-Lat-Vent
Left-Hippocampus
Left-Amygdala
ctx-lh-entorhinal
ctx-lh-middletemporal
ctx-lh-parahippocampal
ctx-lh-superiortemporal


In [770]:
union = set(regions_meg + regions_seeg)

In [771]:
len(interc)

7

In [772]:
len(union)

24

In [773]:
len(interc)/len(union)

0.2916666666666667

In [774]:
diff = set(regions_meg).difference(interc)

In [775]:
len(diff)

10

In [776]:
for elem in sorted(diff):
    print(elem)

Left-Pallidum
Left-VentralDC
Right-Amygdala
Right-Hippocampus
Right-Inf-Lat-Vent
Right-VentralDC
ctx-lh-parsopercularis
ctx-lh-parstriangularis
ctx-lh-precentral
ctx-lh-rostralmiddlefrontal


In [709]:
for vert in iz_seeg:
    if vert in iz_meg:
        interc.append(vert)

for vert in iz_meg:
    if vert in iz_seeg and vert not in interc:
        interc.append(vert)

In [144]:
union = set(iz_seeg + iz_meg)

In [145]:
len(union)

197

In [146]:
sim = len(interc) / len(union)

In [147]:
sim

0.0

In [148]:
len(interc) / len(iz_seeg)

0.0

In [149]:
len(iz_seeg)

17

In [150]:
iz_seeg

[11127,
 12282,
 12317,
 13541,
 13542,
 13606,
 13608,
 13610,
 14723,
 14838,
 14872,
 16161,
 21061,
 26809,
 27396,
 28750,
 30535]